In [1]:
# ----------------------------------------------------------------------------
#
# TITLE - sample_gaia_data.ipynb
# AUTHOR - James Lane
# PROJECT - OHStars
#
# ----------------------------------------------------------------------------
#
# Docstrings and metadata:
'''
Generate samples from 6D phase space taking into account errors and covariances
'''

__author__ = "James Lane"

In [17]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, warnings

## Plotting
from matplotlib import pyplot as plt

## Scipy
from scipy import stats

## Astropy
# from astropy.io import fits
# from astropy.coordinates import SkyCoord
from astropy import table
from astropy import units as apu
from astropy.utils.exceptions import AstropyWarning
# from astropy import wcs

## galpy
# from galpy import orbit
# from galpy import potential
# from galpy.util import bovy_coords as gpcoords
# from galpy.util import bovy_conversion as gpconv
# from galpy.util import bovy_plot as gpplot

# Project-specific imports
sys.path.append('../../src/')
import ohstars.plot

In [18]:
### Change Jupyter settings

from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Markdown, display
from IPython.core.debugger import set_trace

# Matplotlib for notebooks
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

# Print using markdown
def printmd(string, color=None):
    colorstr = "<span style='color:{}'>{}</span>".format(color, string)
    display(Markdown(colorstr))

# Meta
## Future Work

## Information
- Gaia co-variance information: https://www.aanda.org/articles/aa/pdf/2012/07/aa18807-12.pdf

# Read data

In [24]:
# Filename
filename = '../../data/gaia/allstars.vot'

# Read the file
with warnings.catch_warnings():
    warnings.simplefilter('ignore', AstropyWarning)
    tab = table.Table.read(filename)
    
n_stars = len(tab) # 6

# Print a list of the column names
# tab.colnames

['dist',
 'ohstars_oid',
 'col1',
 'col2',
 'col3',
 'col4',
 'solution_id',
 'designation',
 'source_id',
 'random_index',
 'ref_epoch',
 'ra',
 'ra_error',
 'dec',
 'dec_error',
 'parallax',
 'parallax_error',
 'parallax_over_error',
 'pmra',
 'pmra_error',
 'pmdec',
 'pmdec_error',
 'ra_dec_corr',
 'ra_parallax_corr',
 'ra_pmra_corr',
 'ra_pmdec_corr',
 'dec_parallax_corr',
 'dec_pmra_corr',
 'dec_pmdec_corr',
 'parallax_pmra_corr',
 'parallax_pmdec_corr',
 'pmra_pmdec_corr',
 'astrometric_n_obs_al',
 'astrometric_n_obs_ac',
 'astrometric_n_good_obs_al',
 'astrometric_n_bad_obs_al',
 'astrometric_gof_al',
 'astrometric_chi2_al',
 'astrometric_excess_noise',
 'astrometric_excess_noise_sig',
 'astrometric_params_solved',
 'astrometric_primary_flag',
 'astrometric_weight_al',
 'astrometric_pseudo_colour',
 'astrometric_pseudo_colour_error',
 'mean_varpi_factor_al',
 'astrometric_matched_observations',
 'visibility_periods_used',
 'astrometric_sigma5d_max',
 'frame_rotator_object_type',

# Generate Sample
Loop over all stars, assemble the covariance array, and generate a statistical sample

In [25]:
## The radial velocity is from external source and is uncorrelated with gaia data
vrad = np.array([])
vrad_error = np.array([])

In [31]:
# Gaia abbreviations for PS coords, will be used to query table
ps_abbr = np.array(['ra','dec','parallax','pmra','pmdec'])

# Loop over all stars
for i in range(n_stars):
    
    # First gather the means:
    means = np.zeros(5)
    for j in range(5):
        means[j] = tab[i][ps_abbr[j]]
    ###j
    
    # Generate the covaraiance matrix
    cov = np.zeros((5,5))
    for j in range(5):
        for k in range(5):
            if j == k:
                cov[j,k] = tab[i][ps_abbr[j]+'_error']**2
            else:
                try:
                    corr = tab[i][ps_abbr[j]+'_'+ps_abbr[k]+'_corr']
                except KeyError:
                    corr = tab[i][ps_abbr[k]+'_'+ps_abbr[j]+'_corr']
                cov[j,k] = tab[i][ps_abbr[j]+'_error'] * tab[i][ps_abbr[k]+'_error'] * corr
            ##ie
        ###k
    ###j
    
    # Sample from the covariance matrix
    samples = np.random.multivariate_normal( means, cov, size=1000 )
    sample_labels = np.arrays([r'$\alpha$',r'$\delta$',r'$\pi$',r'$\mu_{\alpha}$',r'$\mu_{\delta}$'])
    
    # Create the stair plot
    
    
###i
    
    
   
    

AttributeError: module 'numpy' has no attribute 'arrays'